In [ ]:
!pip install emoji

In [ ]:
import pandas as pd
import numpy as np

import string
import emoji
import nltk
import re

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from collections import Counter
from nltk import pos_tag

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
Data = pd.read_csv('IPHONE_14.csv')

In [ ]:
Data.shape

(1365, 3)

In [ ]:
Data.sample(5)

,Rating,Review,Reviewer
259,5,Very good 💯,Hasan Ali
16,5,"Amazing product, love the design and the feel ...",Anup Singh
574,4,Photo like dslr,Rohit Haral
568,5,All rounder 🔥🔥🔥Thanks for Flipkart 😊😊,Vasim Ansari
513,5,Mind blowing product.....smooth like a butter....,Subhrajit Pal


In [ ]:
Duplicates = Data[Data.duplicated()]
print("Number of duplicates rows : ", Duplicates.shape[0])

Number of duplicates rows :  18


In [ ]:
Data = Data.drop_duplicates()

In [ ]:
MissingValues = Data.isnull().sum()
Missing = (MissingValues / len(Data))*100
print("Percentage of Missing Values : ", Missing)

Percentage of Missing Values :  Rating      0.0
Review      0.0
Reviewer    0.0
dtype: float64


In [ ]:
Contraction = {
    "ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

    "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

    "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

    "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

    "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

    "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

    "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

    "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

    "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

    "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

    "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

    "you're": "you are", "you've": "you have"
}

In [ ]:
StopWords = set(stopwords.words('english'))

def Review_Cleaner(Review):
    newString = Review.lower()

    newString = re.sub(r'\([^)]*\)', '', newString)

    newString = re.sub('"','', newString)

    Words = newString.split(" ")
    Result = []
    for T in Words:
        if T in Contraction:
            Result.append(Contraction[T])
        else:
            Result.append(T)

    newString = ' '.join(Result)

    newString = re.sub(r"'s\b","",newString)

    newString = re.sub("[^a-zA-Z]", " ", newString)

    Tokens = [w for w in newString.split() if not w in StopWords]

    Words = []
    for i in Tokens:
        if len(i) >= 3:
            Words.append(i)

    newString = (" ".join(Words)).strip()

    return newString

In [ ]:
Clean_Review = []
for i in Data['Review']:
    Clean_Review.append(Review_Cleaner(i))

Data['Clean_Review'] = Clean_Review

In [ ]:
Data.sample(5)

,Rating,Review,Reviewer,Clean_Review
1038,5,i like this product because i got it an afford...,Kushal Chauhan,like product got affordable price
396,3,Camera quality very dirty,Ankit Soni,camera quality dirty
933,5,good design good cameragood battery backup bes...,Bharat Adate,good design good cameragood battery backup bes...
459,5,Nice,Shiva Prasad,nice
311,5,Awesome For Night Camera And Day Camera And Pe...,Nizamuddin Seikh,awesome night camera day camera performance


In [ ]:
ReviewsDF = pd.DataFrame({
    'Review': Data['Clean_Review']
})

In [ ]:
DF = pd.DataFrame(ReviewsDF)

def Extract_Dynamic_Features_From_DF(DF):
    reviews = DF['Review'].tolist()
    tokens = [word_tokenize(review) for review in reviews]
    pos_tags = [pos_tag(token) for token in tokens]

    noun_phrases = Counter()
    for pos_tagged_tokens in pos_tags:
        for i in range(len(pos_tagged_tokens) - 1):
            if pos_tagged_tokens[i][1] in ('NN', 'NNS') and pos_tagged_tokens[i + 1][1] in ('NN', 'NNS'):
                noun_phrase = pos_tagged_tokens[i][0] + ' ' + pos_tagged_tokens[i + 1][0]
                noun_phrases[noun_phrase] += 1
            elif pos_tagged_tokens[i][1] in ('NN', 'NNS'):
                noun_phrases[pos_tagged_tokens[i][0]] += 1

    most_common = noun_phrases.most_common(10)
    dynamic_features = [feature[0] for feature in most_common]

    return dynamic_features

Dynamic_Features_DF = Extract_Dynamic_Features_From_DF(DF)

print("Dynamic Features from DataFrame:", Dynamic_Features_DF)

Dynamic Features from DataFrame: ['phone', 'iphone', 'camera', 'product', 'performance', 'quality', 'battery', 'camera quality', 'display', 'apple']


In [ ]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 9.8 MB/s eta 0:00:00


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def Aspect_Sentiment_Analysis_Vader(review, aspect, threshold = -0.10):
    analyzer = SentimentIntensityAnalyzer()
    aspect_lower = aspect.lower()
    aspect_sentiments = []

    sentences = review.split('.')
    for sentence in sentences:
        if aspect_lower in sentence.lower():
            sentiment_score = analyzer.polarity_scores(sentence)['compound']
            aspect_sentiments.append(sentiment_score)

    if aspect_sentiments:
        avg_sentiment = sum(aspect_sentiments) / len(aspect_sentiments)
        if avg_sentiment < -threshold:
            return 'negative'
        elif avg_sentiment > threshold:
            return 'positive'
        else:
            return 'neutral'

    return 'not mentioned'

Sentiment_Data = {feature: [] for feature in Dynamic_Features_DF}

for review in ReviewsDF['Review']:
    for feature in Dynamic_Features_DF:
        sentiment = Aspect_Sentiment_Analysis_Vader(review, feature)
        Sentiment_Data[feature].append(sentiment)

Sentiment_DF = pd.DataFrame(Sentiment_Data)
Result_DF = pd.concat([ReviewsDF, Sentiment_DF], axis=1)

Positive_Counts = Result_DF[Result_DF == 'positive'].count()
Negative_Counts = Result_DF[Result_DF == 'negative'].count()

Total_Counts = Positive_Counts + Negative_Counts
Positive_Percentages = (Positive_Counts / Total_Counts) * 100
Scaled_Positive_Rating = round((Positive_Percentages / 100) * 5, 2)

Result_DF.loc['Positive_Count'] = Positive_Counts
Result_DF.loc['Negative_Count'] = Negative_Counts
Result_DF.loc['Rating'] = Scaled_Positive_Rating
Result_DF

,Review,phone,iphone,camera,product,performance,quality,battery,camera quality,display,apple
0,best smart phone price range compare phones se...,positive,positive,positive,not mentioned,positive,positive,not mentioned,not mentioned,not mentioned,not mentioned
1,nice camera battery drain fast specially video...,not mentioned,not mentioned,positive,not mentioned,not mentioned,not mentioned,positive,not mentioned,not mentioned,not mentioned
2,nice fast,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned
3,god,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned
4,damn great phone camera really good battery la...,positive,positive,positive,not mentioned,not mentioned,not mentioned,positive,not mentioned,positive,positive
...,...,...,...,...,...,...,...,...,...,...,...
1231,NaN,positive,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned
1257,NaN,not mentioned,not mentioned,not mentioned,not mentioned,not mentioned,positive,not mentioned,not mentioned,not mentioned,not mentioned
Positive_Count,0,402,182,244,192,135,125,158,55,71,67
Negative_Count,0,50,26,19,8,12,8,23,3,4,7


In [ ]:
Feature_Ratings = Result_DF.loc['Rating']
Feature_Ratings = Feature_Ratings.drop('Review')
Feature_Ratings

,Rating
phone,4.45
iphone,4.38
camera,4.64
product,4.8
performance,4.59
quality,4.7
battery,4.36
camera quality,4.74
display,4.73
apple,4.53
